In [1]:
# CONTEXTO

# Vamos a ver si podemos mejorar los resultados del modelo LGBM único
# Para ello haremos un stacking, pero no en paralelo, sino en segmentado
# Para evitar segmentarlos según nuestro criterio, pues podríamos sesgar los datos, vamos a seleccionar las n (a definir) variables más importantes
# Pero no podemos utilizar las importancias de nuestro modelo porque se ha entrenado conociendo la variable objetivo, que no sería accesible para datos nuevos

# Por tanto, haremos X clústeres, teniendo en cuenta Y variables (Salvo la objetivo)
# Haremos varias iteraciones con distintas combinaciones y nos quedaremos con el que mejor evaluación tenga

#### Preparativos

In [2]:
# Instalaciones
!pip install lightgbm

In [3]:
!pip install kmodes

In [5]:
# Importaciones

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import KFold, StratifiedKFold, GridSearchCV, cross_val_score, train_test_split
from sklearn.ensemble import StackingClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.preprocessing import StandardScaler

import lightgbm as lgb
from lightgbm import LGBMClassifier

from kmodes.kprototypes import KPrototypes

In [6]:
# Deslimitar/Limitar display Pandas

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 300)
pd.set_option('display.max_colwidth', None)

#### Preparación Dataset

In [33]:
# Read Matches data

df0 = pd.read_csv(r'D:\DEV\Python\00_TFM_PALLADIUM\02_DATASETS_GENERADOS\Reservas_Feature_Engineered_v1.csv', sep = ';', decimal=',')

C:\Users\Diego\AppData\Local\Temp\ipykernel_1192\2151874057.py:3: DtypeWarning: Columns (44) have mixed types. Specify dtype option on import or set low_memory=False.
  df0 = pd.read_csv(r'D:\DEV\Python\00_TFM_PALLADIUM\02_DATASETS_GENERADOS\Reservas_Feature_Engineered_v1.csv', sep = ';', decimal=',')


In [34]:
df0.head()

,ID_RESERVA,ID_HOTEL,HOTEL,LLEGADA,LLEGADA_ANO,LLEGADA_MES,LLEGADA_DIAm,LLEGADA_DIAs,LLEGADA_AVANCE,SALIDA,SALIDA_ANO,SALIDA_MES,SALIDA_DIAm,SALIDA_DIAs,SALIDA_AVANCE,NOCHES,DURACION_ESTANCIA,REGIMEN,ID_TIPO,TIPO,USO,PAX_NUM,PAX_CAT,ADULTOS,NENES,BEBES,ID_CLIENTE,TIPO_CLIENTE,CLIENTE,GRUPO,ID_MONEDA,MONEDA,STATUS,MOTIVO,CHECKIN,SUPLETORIA,CUNAS,FECHA_TOMA,FECHA_TOMA_ANO,FECHA_TOMA_MES,FECHA_TOMA_DIAm,FECHA_TOMA_DIAs,FECHA_TOMA_AVANCE,FECHA_MOD,FECHA_MOD_ANO,FECHA_MOD_MES,FECHA_MOD_DIAm,FECHA_MOD_DIAs,FECHA_MOD_AVANCE,FECHA_CANCELACION,FECHA_CANCELACION_ANO,FECHA_CANCELACION_MES,FECHA_CANCELACION_DIAm,FECHA_CANCELACION_DIAs,FECHA_CANCELACION_AVANCE,LT_TOMA_LLEGADA,LT_TOMA_CANCELACION,ID_FIDELIDAD,FIDELIDAD,VALHAB,VALPEN,VALSERV,VALFIJOS,COMERCIALIZADORA,CMVALHAB,CMVALPEN,CMCVALSERV,VALOR_USD,VALOR_USD_PAX,VALOR_USD_NOCHE,VALOR_USD_PAX_NOCHE,AUTORIZO,GRATIS,PAIS,CONTINENTE,SEGMENTO,FUENTE_NEGOCIO,CANCELADA
0,456094009201,92,GrandPalladiumJamaica&LadyHamiltonResort,2022-12-28,2022,12,28,3,0.989041095890411,2023-01-04,2023,1,4,3,0.00821917808219178,7,Media,AllInclusive,23300,LHAmbassadorSuiteBSOV,2,3,Familias,2,1,0,19686200,1,PALLADIUMTRAVELCLUB_SOCIOS,0,700,USD,1,DESCONOCIDO_0,1,0,0,2022-01-13,2022,1,13,4,0.03287671232876712,2022-12-28,2022,12.0,28.0,3.0,0.989041095890411,NaN,NaT,NaN,NaN,NaN,NaN,349,NaN,0,Ninguna,1868.0,708.0,0.0,0.0,0,1868.0,708.0,0.0,2576.0,858.6666666666666,368.0,122.66666666666666,1,0,ESTADOSUNIDOS,AMERICA,Fixedrates,DIRECTSALES,0
1,425835909201,92,GrandPalladiumJamaica&LadyHamiltonResort,2022-01-22,2022,1,22,6,0.057534246575342465,2022-01-29,2022,1,29,6,0.07671232876712329,7,Media,AllInclusive,43800,LHJuniorSuiteGV,2,3,Familias,2,1,0,19686200,1,PALLADIUMTRAVELCLUB_SOCIOS,0,700,USD,3,DESCONOCIDO_0,0,0,0,2021-07-13,2021,7,13,2,0.5287671232876713,2022-01-20,2022,1.0,20.0,4.0,0.052054794520547946,2022-01-20,2022,1.0,20.0,4.0,0.052054794520547946,193,191.0,0,Ninguna,1329.3,665.0,0.0,0.0,0,1329.3,665.0,0.0,1994.3,664.7666666666667,284.9,94.96666666666667,1,0,ESTADOSUNIDOS,AMERICA,Fixedrates,DIRECTSALES,1
2,549929510601,106,ComplejoRivieraMaya,2023-11-06,2023,11,6,1,0.8465753424657534,2023-11-14,2023,11,14,2,0.8684931506849315,8,Media,AllInclusive,42000,TRSJuniorSuitePS,2,2,Parejas,2,0,0,19686200,1,PALLADIUMTRAVELCLUB_SOCIOS,0,700,USD,3,DESCONOCIDO_0,0,0,0,2023-04-17,2023,4,17,1,0.29041095890410956,2023-11-07,2023,11.0,7.0,2.0,0.8493150684931506,2023-10-17,2023,10.0,17.0,2.0,0.7917808219178082,203,183.0,0,Ninguna,1146.56,1146.56,0.0,12.48,0,1146.56,1146.56,0.0,2305.6,1152.8,288.2,144.1,1,0,CANADA,AMERICA,Fixedrates,DIRECTSALES,1
3,519229110601,106,ComplejoRivieraMaya,2023-11-06,2023,11,6,1,0.8465753424657534,2023-11-23,2023,11,23,4,0.8931506849315068,17,Larga,AllInclusive,42000,TRSJuniorSuitePS,2,2,Parejas,2,0,0,19686200,1,PALLADIUMTRAVELCLUB_SOCIOS,0,700,USD,3,DESCONOCIDO_0,0,0,0,2023-02-22,2023,2,22,3,0.14246575342465753,2023-11-07,2023,11.0,7.0,2.0,0.8493150684931506,2023-09-14,2023,9.0,14.0,4.0,0.7013698630136986,257,204.0,0,Ninguna,2436.44,2436.44,0.0,26.52,0,2436.44,2436.44,0.0,4899.400000000001,2449.7000000000003,288.20000000000005,144.10000000000002,1,0,ESTADOSUNIDOS,AMERICA,Fixedrates,DIRECTSALES,1
4,537405910601,106,ComplejoRivieraMaya,2023-11-25,2023,11,25,6,0.8986301369863013,2023-12-09,2023,12,9,6,0.936986301369863,14,Larga,AllInclusive,13000,COLJuniorSuiteGV,1,1,Single,1,0,0,19686200,1,PALLADIUMTRAVELCLUB_SOCIOS,0,700,USD,3,DESCONOCIDO_0,0,0,0,2023-02-17,2023,2,17,5,0.12876712328767123,2023-11-26,2023,11.0,26.0,7.0,0.9013698630136986,2023-04-11,2023,4.0,11.0,2.0,0.273972602739726,281,53.0,0,Ninguna,2413.18,0.0,0.0,21.84,0,2413.18,0.0,0.0,2435.02,2435.02,173.93,173.93,1,0,RUMANIA,EURESTE,Fixedrates,DIRECTSALES,1


In [35]:
df0.columns

Index(['ID_RESERVA', 'ID_HOTEL', 'HOTEL', 'LLEGADA', 'LLEGADA_ANO',
       'LLEGADA_MES', 'LLEGADA_DIAm', 'LLEGADA_DIAs', 'LLEGADA_AVANCE',
       'SALIDA', 'SALIDA_ANO', 'SALIDA_MES', 'SALIDA_DIAm', 'SALIDA_DIAs',
       'SALIDA_AVANCE', 'NOCHES', 'DURACION_ESTANCIA', 'REGIMEN', 'ID_TIPO',
       'TIPO', 'USO', 'PAX_NUM', 'PAX_CAT', 'ADULTOS', 'NENES', 'BEBES',
       'ID_CLIENTE', 'TIPO_CLIENTE', 'CLIENTE', 'GRUPO', 'ID_MONEDA', 'MONEDA',
       'STATUS', 'MOTIVO', 'CHECKIN', 'SUPLETORIA', 'CUNAS', 'FECHA_TOMA',
       'FECHA_TOMA_ANO', 'FECHA_TOMA_MES', 'FECHA_TOMA_DIAm',
       'FECHA_TOMA_DIAs', 'FECHA_TOMA_AVANCE', 'FECHA_MOD', 'FECHA_MOD_ANO',
       'FECHA_MOD_MES', 'FECHA_MOD_DIAm', 'FECHA_MOD_DIAs', 'FECHA_MOD_AVANCE',
       'FECHA_CANCELACION', 'FECHA_CANCELACION_ANO', 'FECHA_CANCELACION_MES',
       'FECHA_CANCELACION_DIAm', 'FECHA_CANCELACION_DIAs',
       'FECHA_CANCELACION_AVANCE', 'LT_TOMA_LLEGADA', 'LT_TOMA_CANCELACION',
       'ID_FIDELIDAD', 'FIDELIDAD', 'VALHAB', '

In [36]:
# Cuando nos pongamos con modelos que acepten valores categóricos, convendrá pasar algunos int a category
# Ahora lo mantendremos así para poder trabajar con modelos simples

#### Preparación de un DF Mixto

In [37]:
df_mix = df0.copy()

In [38]:
# Reconfigurar dtypes

df_mix.drop(columns=['ID_RESERVA'], inplace=True) # No usar, ID
df_mix.drop(columns=['ID_HOTEL'], inplace=True) # No usar, ID
df_mix['HOTEL'] = df_mix['HOTEL'].astype('category', errors='raise')
df_mix.drop(columns=['LLEGADA'], inplace=True) # No usar, contamina extrapolación
df_mix.drop(columns=['LLEGADA_ANO'], inplace=True) # No usar, contamina extrapolación
df_mix['LLEGADA_MES'] = pd.to_numeric(df_mix['LLEGADA_MES'], errors='raise').astype('category')
df_mix['LLEGADA_DIAm'] = pd.to_numeric(df_mix['LLEGADA_DIAm'], errors='raise').astype('category')
df_mix['LLEGADA_DIAs'] = pd.to_numeric(df_mix['LLEGADA_DIAs'], errors='raise').astype('category')
df_mix['LLEGADA_AVANCE'] = pd.to_numeric(df_mix['LLEGADA_AVANCE'], errors='raise').astype(float)
df_mix.drop(columns=['SALIDA'], inplace=True) # No usar, contamina extrapolación
df_mix.drop(columns=['SALIDA_ANO'], inplace=True) # No usar, contamina extrapolación
df_mix['SALIDA_MES'] = pd.to_numeric(df_mix['SALIDA_MES'], errors='raise').astype('category')
df_mix['SALIDA_DIAm'] = pd.to_numeric(df_mix['SALIDA_DIAm'], errors='raise').astype('category')
df_mix['SALIDA_DIAs'] = pd.to_numeric(df_mix['SALIDA_DIAs'], errors='raise').astype('category')
df_mix['SALIDA_AVANCE'] = pd.to_numeric(df_mix['SALIDA_AVANCE'], errors='raise').astype(float)
df_mix['NOCHES'] = pd.to_numeric(df_mix['NOCHES'], errors='raise').astype('Int64')
df_mix['DURACION_ESTANCIA'] = df_mix['DURACION_ESTANCIA'].astype('category', errors='raise')
df_mix['REGIMEN'] = df_mix['REGIMEN'].astype('category', errors='raise')
df_mix.drop(columns=['ID_TIPO'], inplace=True) # No usar, ID
df_mix['TIPO'] = df_mix['TIPO'].astype('category', errors='raise')
df_mix['USO'] = pd.to_numeric(df_mix['USO'], errors='raise').astype('Int64')
df_mix['PAX_NUM'] = pd.to_numeric(df_mix['PAX_NUM'], errors='raise').astype('Int64')
df_mix['PAX_CAT'] = df_mix['PAX_CAT'].astype('category', errors='raise')
df_mix['ADULTOS'] = pd.to_numeric(df_mix['ADULTOS'], errors='raise').astype('Int64')
df_mix['NENES'] = pd.to_numeric(df_mix['NENES'], errors='raise').astype('Int64')
df_mix['BEBES'] = pd.to_numeric(df_mix['BEBES'], errors='raise').astype('Int64')
df_mix.drop(columns=['ID_CLIENTE'], inplace=True) # No usar, ID
df_mix['TIPO_CLIENTE'] = pd.to_numeric(df_mix['TIPO_CLIENTE'], errors='raise').astype('category')
df_mix['CLIENTE'] = df_mix['CLIENTE'].astype('category', errors='raise')
df_mix['GRUPO'] = pd.to_numeric(df_mix['GRUPO'], errors='raise').astype('category')
df_mix.drop(columns=['ID_MONEDA'], inplace=True) # No usar, ID
df_mix['MONEDA'] = df_mix['MONEDA'].astype('category', errors='raise')
df_mix.drop(columns=['STATUS'], inplace=True) # No usar, redundante con variable objetivo
df_mix.drop(columns=['MOTIVO'], inplace=True) # No usar, imposible ver el futuro
df_mix.drop(columns=['CHECKIN'], inplace=True) # No usar, imposible ver el futuro
df_mix['SUPLETORIA'] = pd.to_numeric(df_mix['SUPLETORIA'], errors='raise').astype('Int64')
df_mix['CUNAS'] = pd.to_numeric(df_mix['CUNAS'], errors='raise').astype('Int64')
df_mix.drop(columns=['FECHA_TOMA'], inplace=True) # No usar, contamina extrapolación
df_mix.drop(columns=['FECHA_TOMA_ANO'], inplace=True) # No usar, contamina extrapolación
df_mix['FECHA_TOMA_MES'] = pd.to_numeric(df_mix['FECHA_TOMA_MES'], errors='raise').astype('category')
df_mix['FECHA_TOMA_DIAm'] = pd.to_numeric(df_mix['FECHA_TOMA_DIAm'], errors='raise').astype('category')
df_mix['FECHA_TOMA_DIAs'] = pd.to_numeric(df_mix['FECHA_TOMA_DIAs'], errors='raise').astype('category')
df_mix['FECHA_TOMA_AVANCE'] = pd.to_numeric(df_mix['FECHA_TOMA_AVANCE'], errors='raise').astype(float)
df_mix.drop(columns=['FECHA_MOD'], inplace=True) # No usar, contamina extrapolación, imposible ver el futuro
df_mix.drop(columns=['FECHA_MOD_ANO'], inplace=True) # No usar, contamina extrapolación, imposible ver el futuro
df_mix.drop(columns=['FECHA_MOD_MES'], inplace=True) # No usar, imposible ver el futuro
df_mix.drop(columns=['FECHA_MOD_DIAm'], inplace=True) # No usar, imposible ver el futuro
df_mix.drop(columns=['FECHA_MOD_DIAs'], inplace=True) # No usar, imposible ver el futuro
df_mix.drop(columns=['FECHA_MOD_AVANCE'], inplace=True) # No usar, imposible ver el futuro
df_mix.drop(columns=['FECHA_CANCELACION'], inplace=True) # No usar, contamina extrapolación, imposible ver el futuro
df_mix.drop(columns=['FECHA_CANCELACION_ANO'], inplace=True) # No usar, contamina extrapolación, imposible ver el futuro
df_mix.drop(columns=['FECHA_CANCELACION_MES'], inplace=True) # No usar, imposible ver el futuro
df_mix.drop(columns=['FECHA_CANCELACION_DIAm'], inplace=True) # No usar, imposible ver el futuro
df_mix.drop(columns=['FECHA_CANCELACION_DIAs'], inplace=True) # No usar, imposible ver el futuro
df_mix.drop(columns=['FECHA_CANCELACION_AVANCE'], inplace=True) # No usar, imposible ver el futuro
df_mix['LT_TOMA_LLEGADA'] = pd.to_numeric(df_mix['LT_TOMA_LLEGADA'], errors='raise').astype('Int64')
df_mix.drop(columns=['LT_TOMA_CANCELACION'], inplace=True) # No usar, imposible ver el futuro
df_mix.drop(columns=['ID_FIDELIDAD'], inplace=True) # No usar, ID
df_mix['FIDELIDAD'] = df_mix['FIDELIDAD'].astype('category', errors='raise')
df_mix.drop(columns=['VALHAB'], inplace=True) # No usar, no está convertido y teniendo el valor total es redundante y/o dependiente de COMERCIALIZADORA
df_mix.drop(columns=['VALPEN'], inplace=True) # No usar, no está convertido y teniendo el valor total es redundante y/o dependiente de COMERCIALIZADORA
df_mix.drop(columns=['VALSERV'], inplace=True) # No usar, no está convertido y teniendo el valor total es redundante y/o dependiente de COMERCIALIZADORA
df_mix.drop(columns=['VALFIJOS'], inplace=True) # No usar, no está convertido y teniendo el valor total es redundante y/o dependiente de COMERCIALIZADORA
df_mix['COMERCIALIZADORA'] = pd.to_numeric(df_mix['COMERCIALIZADORA'], errors='raise').astype('category')
df_mix.drop(columns=['CMVALHAB'], inplace=True) # No usar, no está convertido y teniendo el valor total es redundante y/o dependiente de COMERCIALIZADORA
df_mix.drop(columns=['CMVALPEN'], inplace=True) # No usar, no está convertido y teniendo el valor total es redundante y/o dependiente de COMERCIALIZADORA
df_mix.drop(columns=['CMCVALSERV'], inplace=True) # No usar, no está convertido y teniendo el valor total es redundante y/o dependiente de COMERCIALIZADORA
df_mix['VALOR_USD'] = pd.to_numeric(df_mix['VALOR_USD'], errors='raise').astype(float)
df_mix['VALOR_USD_PAX'] = pd.to_numeric(df_mix['VALOR_USD_PAX'], errors='raise').astype(float)
df_mix['VALOR_USD_NOCHE'] = pd.to_numeric(df_mix['VALOR_USD_NOCHE'], errors='raise').astype(float)
df_mix['VALOR_USD_PAX_NOCHE'] = pd.to_numeric(df_mix['VALOR_USD_PAX_NOCHE'], errors='raise').astype(float)
df_mix.drop(columns=['AUTORIZO'], inplace=True) # No usar, de momento no muy claro
df_mix['GRATIS'] = pd.to_numeric(df_mix['GRATIS'], errors='raise').astype('category')
df_mix['PAIS'] = df_mix['PAIS'].astype('category', errors='raise')
df_mix['CONTINENTE'] = df_mix['CONTINENTE'].astype('category', errors='raise')
df_mix['SEGMENTO'] = df_mix['SEGMENTO'].astype('category', errors='raise')
df_mix['FUENTE_NEGOCIO'] = df_mix['FUENTE_NEGOCIO'].astype('category', errors='raise')
df_mix['CANCELADA'] = pd.to_numeric(df_mix['CANCELADA'], errors='raise').astype('category')

In [39]:
df_mix.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1139289 entries, 0 to 1139288
Data columns (total 42 columns):
 #   Column               Non-Null Count    Dtype   
---  ------               --------------    -----   
 0   HOTEL                1139289 non-null  category
 1   LLEGADA_MES          1139289 non-null  category
 2   LLEGADA_DIAm         1139289 non-null  category
 3   LLEGADA_DIAs         1139289 non-null  category
 4   LLEGADA_AVANCE       1139289 non-null  float64 
 5   SALIDA_MES           1139289 non-null  category
 6   SALIDA_DIAm          1139289 non-null  category
 7   SALIDA_DIAs          1139289 non-null  category
 8   SALIDA_AVANCE        1139289 non-null  float64 
 9   NOCHES               1139289 non-null  Int64   
 10  DURACION_ESTANCIA    1139289 non-null  category
 11  REGIMEN              1139289 non-null  category
 12  TIPO                 1139289 non-null  category
 13  USO                  1139289 non-null  Int64   
 14  PAX_NUM              1139289 non-n

In [40]:
X_mix = df_mix.drop(columns=['CANCELADA'])

y_mix = df_mix['CANCELADA']

X_mix_train, X_mix_test, y_mix_train, y_mix_test = train_test_split(X_mix, y_mix, test_size=0.2, random_state=42)

# El test no lo tocaremos nunca hasta el final

#### Clusterización no supervisada (Prueba)

In [46]:
df_kproto = df_mix.copy()

# Cortamos el DF con prueba y error
# Lo suficientemente grande como para no perder demasiada información
# Lo suficientemente pequeño para que al realizar el bucle de optimización no estemos computando hasta el mes que viene
df_kproto = df_kproto.sample(n=100000, random_state=42)

# Para quitar CANCELADA
# Porque cuando clustericemos los nuevos datos no debemos contaminarlos con información futura
df_kproto = df_kproto.drop(columns='CANCELADA')

# Tipos de variables
num_cols = df_mix.select_dtypes(include='number').columns
cat_cols = df_kproto.select_dtypes(include='category').columns

In [47]:
# Estandarizamos numéricas
df_kproto[num_cols] = StandardScaler().fit_transform(df_kproto[num_cols])

In [48]:
df_kproto.head()

,HOTEL,LLEGADA_MES,LLEGADA_DIAm,LLEGADA_DIAs,LLEGADA_AVANCE,SALIDA_MES,SALIDA_DIAm,SALIDA_DIAs,SALIDA_AVANCE,NOCHES,DURACION_ESTANCIA,REGIMEN,TIPO,USO,PAX_NUM,PAX_CAT,ADULTOS,NENES,BEBES,TIPO_CLIENTE,CLIENTE,GRUPO,MONEDA,SUPLETORIA,CUNAS,FECHA_TOMA_MES,FECHA_TOMA_DIAm,FECHA_TOMA_DIAs,FECHA_TOMA_AVANCE,LT_TOMA_LLEGADA,FIDELIDAD,COMERCIALIZADORA,VALOR_USD,VALOR_USD_PAX,VALOR_USD_NOCHE,VALOR_USD_PAX_NOCHE,GRATIS,PAIS,CONTINENTE,SEGMENTO,FUENTE_NEGOCIO
1021862,TRSCapCana,9,4,1,0.576887,9,9,6,0.644448,-0.102563,Corta,AllInclusive,CAPJuniorSuiteSWPS,0.357463,-0.316858,Parejas,0.011630,-0.430247,-0.162304,1,HOTELCOLLECT_EXPEDIA,0,USD,-0.013418,-0.183173,7,11,2,0.182664,-0.566171,Ninguna,0,0.200407,0.294547,0.417265,0.503937,0,ESTADOSUNIDOS,AMERICA,BAR,E-COMMERCE
912172,ComplejoRivieraMaya,1,2,7,-1.794011,1,5,3,-1.745244,-0.734028,Corta,AllInclusive,COLDeluxeGardenView,0.357463,1.052331,Familias,2.107098,-0.430247,-0.162304,1,COSTAMARTRAVELCRUISE&TOURSINC,0,USD,-0.013418,-0.183173,11,18,4,1.447940,-0.647379,Ninguna,1,-0.439444,-0.672072,0.256742,-0.340604,0,PERU,AMERICA,FixedRates,T.O./T.A.
391766,TRSCapCana,3,24,4,-1.010163,3,25,5,-0.980930,-1.365494,Corta,AllInclusive,CAPJuniorSuiteMV,0.357463,-0.316858,Parejas,0.011630,-0.430247,-0.162304,1,AGODACOMPANYPTELTD,0,USD,-0.013418,-0.183173,3,21,1,-0.907420,-0.988451,Ninguna,0,-1.039913,-1.026059,-0.241495,-0.149834,0,SINPAIS,DESCONOCIDO,BAR,E-COMMERCE
564572,ComplejoPuntaCana,7,23,6,0.160770,7,28,4,0.228429,-0.102563,Corta,AllInclusive,TRSJuniorSuitePS,0.357463,-0.316858,Parejas,0.011630,-0.430247,-0.162304,1,ROIBACK(GLOBALOBIS.L.),0,USD,-0.013418,-0.183173,4,10,7,-0.712762,-0.168254,PalladiumRewards,0,0.049616,0.133995,0.184173,0.272610,0,FRANCIA,EUROPA,Loyalty,DIRECTSALES
761584,GrandPalladiumImbassaiResort&Spa,9,8,5,0.615595,9,12,2,0.673472,-0.418295,Corta,AllInclusive,SSAJuniorSuitePS,-2.701178,-1.686047,Single,-2.083839,-0.430247,-0.162304,1,ROIBACK(GLOBALOBIS.L.),0,BRL,-0.013418,-0.183173,12,12,1,1.681529,1.179793,PalladiumRewards,0,-0.655359,0.009546,-0.678307,0.544395,0,BRASIL,AMERICA,Loyalty,DIRECTSALES


In [49]:
# El algoritmo que utilizaremos necesitará los índices. Así que:
cat_idx = [df_mix.columns.get_loc(col) for col in cat_cols]

In [50]:
# Clusterizador
# Hemos escogido K-Prototypes porque puede manejar datos categóricos y tenemos muchos
# Vamos a ver cómo funciona para posteriormente optimizarlo

# cat_cols a string para que el algoritmo no se queje
df_kproto[cat_cols] = df_kproto[cat_cols].astype(str)

# Convertimos a matriz numpy
X = df_kproto.to_numpy()

In [52]:
# Entrena el modelo
kproto = KPrototypes(n_clusters=3, init='Cao', n_init=1, verbose=1, random_state=42)
clusters = kproto.fit_predict(X, categorical=cat_idx)

# Añade los clusters al DataFrame
df_kproto['cluster'] = clusters

Initialization method and algorithm are deterministic. Setting n_init to 1.
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run: 1, iteration: 1/100, moves: 12967, ncost: 2000507.392767559
Run: 1, iteration: 2/100, moves: 7201, ncost: 1992901.8966219164
Run: 1, iteration: 3/100, moves: 3943, ncost: 1990472.702493476
Run: 1, iteration: 4/100, moves: 2231, ncost: 1989767.690989339
Run: 1, iteration: 5/100, moves: 1219, ncost: 1989560.8209572171
Run: 1, iteration: 6/100, moves: 780, ncost: 1989475.3066277078
Run: 1, iteration: 7/100, moves: 463, ncost: 1989440.700031424
Run: 1, iteration: 8/100, moves: 269, ncost: 1989427.8891032618
Run: 1, iteration: 9/100, moves: 181, ncost: 1989423.5505020528
Run: 1, iteration: 10/100, moves: 82, ncost: 1989422.6642098778
Run: 1, iteration: 11/100, moves: 48, ncost: 1989422.3228392499
Run: 1, iteration: 12/100, moves: 42, ncost: 1989422.0544932005
Run: 1, iteration: 13/100, moves: 35, ncost: 1989421.8609295753
Run: 1, it

In [53]:
# Ojo, vamos a ver si los clústeres estarían balanceados, aunque fuera mínimamente
df_kproto['cluster'].value_counts()

cluster
0    55448
1    31231
2    13321
Name: count, dtype: int64

#### Obtención de las n variables más importantes

In [54]:
def variable_importance_proxy(df, cat_cols, num_cols, n_clusters=3, random_state=42):
    print(f"Columnas iniciales: {len(df.columns)}")
    cols = list(df.columns)
    
    # Índices categóricos en todas las columnas
    cat_idx = [df.columns.get_loc(col) for col in cat_cols if col in cols]
    
    # Clustering con todas las variables
    X = df.to_numpy()
    kproto_full = KPrototypes(n_clusters=n_clusters, init='Cao', n_init=1, verbose=0, random_state=random_state)
    kproto_full.fit_predict(X, categorical=cat_idx)
    full_cost = kproto_full.cost_
    print(f"Costo con todas las variables: {full_cost:.2f}\n")
    
    importance = {}
    for c in cols:
        print(f"Probando sin columna: {c}")
        cols_temp = [col for col in cols if col != c]
        df_temp = df[cols_temp]
        cat_cols_temp = [col for col in cat_cols if col in cols_temp]
        cat_idx_temp = [df_temp.columns.get_loc(col) for col in cat_cols_temp]
        
        X_temp = df_temp.to_numpy()
        kproto_temp = KPrototypes(n_clusters=n_clusters, init='Cao', n_init=1, verbose=0, random_state=random_state)
        kproto_temp.fit_predict(X_temp, categorical=cat_idx_temp)
        cost_temp = kproto_temp.cost_
        
        importance[c] = cost_temp - full_cost  # diferencia de costo al eliminar c
    
    # Ordenar variables por impacto en el costo al eliminarlas (mayor impacto = más importante)
    importance_sorted = dict(sorted(importance.items(), key=lambda item: item[1], reverse=True))
    
    print("\nImpacto en costo al eliminar cada variable:")
    for var, imp in importance_sorted.items():
        print(f"{var}: {imp:.2f}")
    
    return importance_sorted

In [32]:
# Usalo así:
variable_importance_proxy(df_kproto, cat_cols, num_cols, n_clusters=3, random_state=42)

Columnas iniciales: 41
Costo con todas las variables: 1989421.61

Probando sin columna: HOTEL
Probando sin columna: LLEGADA_MES
Probando sin columna: LLEGADA_DIAm
Probando sin columna: LLEGADA_DIAs
Probando sin columna: LLEGADA_AVANCE
Probando sin columna: SALIDA_MES
Probando sin columna: SALIDA_DIAm
Probando sin columna: SALIDA_DIAs
Probando sin columna: SALIDA_AVANCE
Probando sin columna: NOCHES
Probando sin columna: DURACION_ESTANCIA
Probando sin columna: REGIMEN
Probando sin columna: TIPO
Probando sin columna: USO
Probando sin columna: PAX_NUM
Probando sin columna: PAX_CAT
Probando sin columna: ADULTOS
Probando sin columna: NENES
Probando sin columna: BEBES
Probando sin columna: TIPO_CLIENTE
Probando sin columna: CLIENTE
Probando sin columna: GRUPO
Probando sin columna: MONEDA
Probando sin columna: SUPLETORIA
Probando sin columna: CUNAS
Probando sin columna: FECHA_TOMA_MES
Probando sin columna: FECHA_TOMA_DIAm
Probando sin columna: FECHA_TOMA_DIAs
Probando sin columna: FECHA_TOMA_A

{'GRATIS': np.float64(-33.0),
 'TIPO_CLIENTE': np.float64(-974.9999999995343),
 'REGIMEN': np.float64(-2030.9999999995343),
 'FIDELIDAD': np.float64(-6880.425789162982),
 'GRUPO': np.float64(-7112.999999999534),
 'CONTINENTE': np.float64(-8598.903194267768),
 'MONEDA': np.float64(-8977.631639320869),
 'PAX_CAT': np.float64(-13350.910761601524),
 'DURACION_ESTANCIA': np.float64(-15612.036344724474),
 'COMERCIALIZADORA': np.float64(-20790.136955805123),
 'SEGMENTO': np.float64(-27656.77462804923),
 'FUENTE_NEGOCIO': np.float64(-28063.51357418415),
 'PAIS': np.float64(-32679.653382613556),
 'HOTEL': np.float64(-37341.50589365256),
 'SALIDA_DIAs': np.float64(-38530.987921290565),
 'LLEGADA_DIAs': np.float64(-39886.50010474818),
 'FECHA_TOMA_DIAs': np.float64(-41155.043643824756),
 'SALIDA_MES': np.float64(-42848.49645147659),
 'LLEGADA_MES': np.float64(-43238.20763162244),
 'FECHA_TOMA_MES': np.float64(-44071.65665214672),
 'CLIENTE': np.float64(-46170.71254212735),
 'TIPO': np.float64(-46

In [ ]:
# Ahora recortamos las variables menos importantes
# Idealmente debería hacerse 1 a 1 y recalcular todas las importancias, pero dado que la anterior celda tardó 138min en ejecutarse, no es una opción
# Tendremos que quitar unas 10 de golpe y ver qué tal reaccionan las métricas

df_kproto = df_kproto[[
    'GRATIS',
    'TIPO_CLIENTE',
    'REGIMEN',
    'FIDELIDAD',
    'GRUPO',
    'CONTINENTE',
    'MONEDA',
    'PAX_CAT',
    'DURACION_ESTANCIA',
    'COMERCIALIZADORA',
    'SEGMENTO',
    'FUENTE_NEGOCIO',
    'PAIS',
    'HOTEL',
    'SALIDA_DIAs',
    'LLEGADA_DIAs',
    'FECHA_TOMA_DIAs',
    'SALIDA_MES',
    'LLEGADA_MES',
    'FECHA_TOMA_MES',
    'CLIENTE',
    'TIPO',
    'LLEGADA_DIAm',
    'SALIDA_DIAm',
    'FECHA_TOMA_DIAm',
    'PAX_NUM',
    'VALOR_USD_PAX',
    'VALOR_USD_PAX_NOCHE',
    'VALOR_USD',
    'VALOR_USD_NOCHE',
    'SALIDA_AVANCE',
    'NOCHES',
    'LLEGADA_AVANCE',
    'ADULTOS',
    'CUNAS',
    'BEBES',
    'USO',
    'FECHA_TOMA_AVANCE',
    'LT_TOMA_LLEGADA',
    'SUPLETORIA',
    'NENES',
]]

In [57]:
df_kproto.info()

<class 'pandas.core.frame.DataFrame'>
Index: 100000 entries, 1021862 to 887948
Data columns (total 41 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   GRATIS               100000 non-null  object 
 1   TIPO_CLIENTE         100000 non-null  object 
 2   REGIMEN              100000 non-null  object 
 3   FIDELIDAD            100000 non-null  object 
 4   GRUPO                100000 non-null  object 
 5   CONTINENTE           100000 non-null  object 
 6   MONEDA               100000 non-null  object 
 7   PAX_CAT              100000 non-null  object 
 8   DURACION_ESTANCIA    100000 non-null  object 
 9   COMERCIALIZADORA     100000 non-null  object 
 10  SEGMENTO             100000 non-null  object 
 11  FUENTE_NEGOCIO       100000 non-null  object 
 12  PAIS                 100000 non-null  object 
 13  HOTEL                100000 non-null  object 
 14  SALIDA_DIAs          100000 non-null  object 
 15  LLEGADA_DIAs    

In [ ]:
# Sospechoso que TODAS las categóricas sean más importantes que TODAS las numéricas
# Por lo visto hay que tunear el parámetro gamma
# Visto lo que tarda cada teración y, recordemos que estamos clusterizando con el 10% de los datos... No es una opción con nuestros humildes medios
# Pasamos a una clusterización con las variables que más sentido tengan desde el punto de vista de negocio